# Solving By Searching
## The Problems
Code up the Prisoners and Guards problem in the book, using the python code provided at the AIMA github site.  
In addition, code up the 8-puzzle, in this case it makes sense to use heuristics to speed up the search.

## What I Know
I am familiar with both puzzles and how to represent theme but I need to learn how to program a solution using a breadth first approach.

### The Prisoner Guard/ Canibals and Missionaries Problem
In the missionaries and cannibals problem, three missionaries and three cannibals must cross a river using a boat which can carry at most two people, under the constraint that, for both banks, if there are missionaries present on the bank, they cannot be outnumbered by cannibals (if they were, the cannibals would eat the missionaries) [wiki](https://en.wikipedia.org/wiki/Missionaries_and_cannibals_problem)

In [2]:
# Imports needed to run the code in this notebook
import numpy as np
import collections
import random

In [ ]:
def actions( state):
    # possible
    possible = np.array([[0, 1, 1], [1, 0, 1], [1, 1, 1], [0, 2, 1], [2, 0, 1]])
    sending = 1 - 2 * state[2] # either -1 or 1
    for p in possible:
        # multiplies the possible action by the status of the boat and adds that to the sate
        ns_ws = np.add(np.multiply(sending, p), state)
        # subtracts the ns from the original state showing the other side of the shore
        ns_rs = np.subtract(np.array([3, 3, 1]), ns_ws)
        # makes sure we dont get negative prisoners
        if ns_ws[0] <0:
            continue
        if ns_rs[0] <0:
            continue
        # makes sure we dont get more than the initial 3
        if ns_ws[1]>3:
            continue
        if ns_rs[1] >3:
            continue
        # checks to make sure the prisoners out number the guards
        if ns_ws[1] > ns_ws[0] and ns_ws[0] > 0:
            continue
        if ns_rs[1] > ns_rs[0] and ns_rs[0] > 0:
            continue
        yield ns_ws
    pass


def is_in(item,things):
    return any((item == athing).all() for athing in things)

def breadth_first_search():
    node = np.array([3,3,1]) # starting point
    goal = np.array([0,0,0]) # goal
    frontier = collections.deque([node])
    explored = []
    print(frontier)
    if (node == goal).all():
        return True
    while True:
        try:
            node = frontier.pop()
        except IndexError: 
            return False

        explored.append(node)
        # Formatted output
        for x in explored:
            print(x)
        print("~~~~~~~~~~")
        

        for newstate in  actions(node):
            if not(is_in(newstate,explored) or is_in(newstate,frontier)):

                if (newstate == goal).all():
                    return True
                frontier.append(newstate)



print(breadth_first_search())

### The Eight Puzzle
The eight puzzle is a sliding puzzle which is a simplied version of the 15 puzzle [wiki](https://en.wikipedia.org/wiki/15_puzzle) In which we will create a solvable puzzle by applying 'random' moves to the board from a solvd state the program will than preform a search with heuristics to solve the puzzle. The first is to make a function which applies an action to the baord.

In [3]:
def apply_action(board, action):
    """
    Applies an action to a board
    :param board:  the board which the action is being applied to
    :param action: a integer from 0-3 which corresponds to actions: 0: up, 1:right, 2:down, 3:left
    :return: the board with the new acction applied to it
    """
    deltas = np.array([[-1, 0, 1, 0], [0, 1, 0, -1]])  # changes based on what direction the tile is moved
    posx, posy = np.where(np.isin(board, [0]))  # grabs the position of the blank tile
    (x, y) = (posx[0], posy[0])
    (new_x, new_y) = (x + deltas[0, action], y + deltas[1, action])
    if new_x < 0 or new_y < 0:  # necessary because negative numbers wrap arround
        new_x = 3
    try:
        el = board[new_x, new_y]
        board[x, y] = el
        board[new_x, new_y] = 0
    except IndexError:
        # print('index error')
        pass
    return board

The next bit is to add a function which messes up the board and checks to see if it matches the goal

In [4]:
def mess_up(board, moves):
    """
    Messes up the the board using number of moves
    :param board: The board which needs to be messed up
    :param moves:  Number of moves to mess up the board with
    :return:
    """
    for i in range(0, moves):
        board = apply_action(board, random.randint(0, 3))  # generates a random move based on the comment at the top
    pass


def goal_test(board):
    """
    Tests the board to see if it matches the goal state
    :param board: the board which we want to tesy
    :return: true if the board is the same as the goal and fallse if not
    """
    goal = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 0]])
    return np.array_equal(board, goal)

next we need to make a function which finds the solution through random search. 

In [5]:
def random_search(board):
    """
    create a sequence of 32 random  moves and keep going until it is solved. this may take a long time to return
    :param board:
    :return:
    """
    actions = ["up", "right", "down", "left"]  # The  moves mapped to an array to write the moves to an array for
    # printing
    while True:
        sequence = []
        new_board = np.copy(board)
        c_board = np.copy(board)  # a board to compare to
        for iteration in range(32):
            r = random.randint(0, 3)
            action = actions[r]
            new_board = apply_action(new_board, r)
            if not np.array_equal(new_board, c_board):  # checks to see if the move does anything to the board
                # if there is a change it adds the move to the sequence
                sequence.append(action)
                c_board = np.copy(new_board)  # copies the new board to the compare board
            if goal_test(new_board):
                print(new_board)
                return sequence
    pass

Next lets test our random search by creating a board which we mess up with 50 random moves and then solve with a random search

In [9]:
board = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 0]])
print(goal_test(board))
mess_up(board, 50)
print('Starting Board')
print(board)
print('Goal Test: %s' % goal_test(board))
print(random_search(board))

True
Starting Board
[[2 0 8]
 [1 6 3]
 [7 4 5]]
Goal Test: False
[[1 2 3]
 [4 5 6]
 [7 8 0]]
['right', 'down', 'left', 'up', 'right', 'down', 'left', 'down', 'right', 'up', 'down', 'up', 'up', 'left', 'left', 'down', 'right', 'down', 'up', 'down', 'left', 'right', 'right']


In [10]:
def n_out_of_order(board):
    goal = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 0]])
    return np.count_nonzero(np.subtract(board,goal))


def manhattan(board):
    """
    grabs the total manhattan distance of the board
    """
    goal = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 0]])
    td = 0
    for item in [1, 2, 3, 4, 5, 6, 7, 8]:
        gx, gy = np.where(np.isin(goal, item))
        cx, cy = np.where(np.isin(board, item))
        td += abs(gx-cx)+abs(gy-cy)
    return td[0]


def max_heuristic(board):
    score1 = manhattan(board)
    score2 = n_out_of_order(board)
    return max(score1, score2)

### Conclusion
When I was coding and trying to figure out the search for the 8 puzzle I had run into several problems. The first and largest problem was that the puzzle was wrapping around itself. An example of this was with the following board state:


821

570

436 


if we moved the open space from the current position to the right it would go around to the other side making the new board state:


821

057

436


this was a simple solution which was added and commented above.
